<a href="https://colab.research.google.com/github/mattharper777/learning/blob/main/TallestTreeInTheWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent LLM Deliberation System
A notebook for running multiple LLMs with opposing personas to analyze topics
Optimized for Google Colaboratory

In [31]:
# =============================================================================
# CELL 1: Install Dependencies
# =============================================================================
"""
Run this cell first to install required packages.
This will take about 30-60 seconds.
"""

!pip install -q litellm pandas numpy tabulate openai anthropic google-generativeai

import litellm
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import json
import time
from collections import Counter
import os
from getpass import getpass

# Suppress litellm warnings
litellm.suppress_debug_info = True

print("✓ Dependencies installed and loaded successfully")
print("✓ Ready to configure API keys in the next cell")

✓ Dependencies installed and loaded successfully
✓ Ready to configure API keys in the next cell


In [32]:
# =============================================================================
# CELL 2: Configure API Keys (SECURE METHOD)
# =============================================================================
"""
SECURE API KEY INPUT FOR GOOGLE COLAB

This cell provides secure input for your API keys. They won't be visible after entry.
You only need keys for the providers you want to use.

Get API keys from:
- OpenAI: https://platform.openai.com/api-keys
- Anthropic: https://console.anthropic.com/
- Google AI: https://makersuite.google.com/app/apikey

OPTION 1: Use Google Colab Secrets (Recommended for repeated use)
1. Click the key icon (🔑) in the left sidebar
2. Add secrets named: OPENAI_API_KEY, ANTHROPIC_API_KEY, GOOGLE_API_KEY
3. Enable "Notebook access" for each
4. Uncomment the "Using Colab Secrets" section below

OPTION 2: Enter keys manually each time (Secure, one-time use)
The default option below - keys won't be saved
"""

# Choose your method by uncommenting ONE of the following sections:

# --- METHOD 1: Using Colab Secrets (Uncomment these 4 lines) ---
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# --- METHOD 2: Manual Entry (Default - Currently Active) ---
#print("🔐 Enter your API keys securely (input will be hidden)")
#print("Press Enter to skip any provider you don't want to use\n")

#openai_key = getpass("OpenAI API Key (or press Enter to skip): ")
#anthropic_key = getpass("Anthropic API Key (or press Enter to skip): ")
#google_key = getpass("Google API Key (or press Enter to skip): ")

# The following lines are removed as they refer to 'openai_key', 'anthropic_key', 'google_key' which are not defined
# when using Colab Secrets, and are redundant if manually entered.
# if openai_key:
#     os.environ["OPENAI_API_KEY"] = openai_key
# if anthropic_key:
#     os.environ["ANTHROPIC_API_KEY"] = anthropic_key
# if google_key:
#     os.environ["GOOGLE_API_KEY"] = google_key

# Configure which models to use based on available keys from environment variables
MODELS = []

if os.environ.get("OPENAI_API_KEY"):
    MODELS.append("gpt-4o-mini")  # Cost-effective OpenAI model
    # MODELS.append("gpt-4o")  # Uncomment for more powerful (but expensive) model

if os.environ.get("ANTHROPIC_API_KEY"):
    MODELS.append("claude-3-5-haiku-20241022")  # Fast and efficient
    # MODELS.append("claude-3-5-sonnet-20241022")  # Uncomment for more powerful model

if os.environ.get("GOOGLE_API_KEY"):
    MODELS.append("gemini-1.5-flash")  # Fast and free tier available
    # MODELS.append("gemini-1.5-pro")  # Uncomment for more powerful model

if not MODELS:
    print("\n⚠️  WARNING: No API keys provided! Please run this cell again with at least one API key.")
else:
    print(f"\n✓ Configured {len(MODELS)} models:")
    for model in MODELS:
        print(f"  • {model}")
    print("\n✓ Ready to proceed to the next cell!")


✓ Configured 3 models:
  • gpt-4o-mini
  • claude-3-5-haiku-20241022
  • gemini-1.5-flash

✓ Ready to proceed to the next cell!


In [33]:
# =============================================================================
# CELL 3: Define the Topic and Personas
# =============================================================================
"""
Define what topic you want the agents to debate and what personas they should adopt.
"""

# The topic/question for analysis
TOPIC = "What is the tallest tree that ever existed?"

# Define opposing personas
PERSONAS = {
    "optimist": {
        "description": "An enthusiastic advocate who sees the benefits and opportunities",
        "instruction": "Argue strongly in favor of this position. Focus on benefits, opportunities, and positive outcomes."
    },
    "skeptic": {
        "description": "A critical skeptic who identifies risks and problems",
        "instruction": "Argue against this position. Focus on risks, downsides, and potential negative consequences."
    },
    "pragmatist": {
        "description": "A balanced, practical analyst",
        "instruction": "Provide a balanced, practical analysis. Consider both benefits and challenges realistically."
    }
}

print(f"✓ Topic: {TOPIC}")
print(f"✓ Personas: {list(PERSONAS.keys())}")

✓ Topic: What is the tallest tree that ever existed?
✓ Personas: ['optimist', 'skeptic', 'pragmatist']


In [38]:
def call_llm(model: str, prompt: str, max_retries: int = 3) -> str:
    """Call an LLM with retry logic."""
    for attempt in range(max_retries):
        try:
            if model.startswith("gemini"):
                api_key_to_use = os.environ.get("GOOGLE_API_KEY")
                if not api_key_to_use:
                    return "ERROR: GOOGLE_API_KEY not found in environment variables."

                # Explicitly set litellm's API key for Gemini models
                litellm.api_key['gemini'] = api_key_to_use

                response = litellm.completion(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7
                )
            else:
                response = litellm.completion(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7
                )
            return response.choices[0].message.content
        except Exception as e:
            if attempt == max_retries - 1:
                return f"ERROR: {str(e)}"
            time.sleep(2 ** attempt)  # Exponential backoff


def extract_json_from_response(response: str) -> Dict:
    """Extract JSON from LLM response, handling markdown code blocks."""
    try:
        # Try direct JSON parse first
        return json.loads(response)
    except:
        # Try to find JSON in markdown code blocks
        import re
        json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', response, re.DOTALL)
        if json_match:
            return json.loads(json_match.group(1))
        # Try to find JSON without code blocks
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            return json.loads(json_match.group(0))
        raise ValueError("Could not extract JSON from response")


print("✓ Helper functions defined")

✓ Helper functions defined


In [39]:
# =============================================================================
# CELL 5: Stage 1 - Generate Arguments from Different Personas
# =============================================================================
"""
Each model generates arguments from each persona's perspective.
"""

print("=" * 80)
print("STAGE 1: GENERATING ARGUMENTS FROM DIFFERENT PERSONAS")
print("=" * 80)

arguments = {}

for model in MODELS:
    print(f"\n🤖 Querying {model}...")
    arguments[model] = {}

    for persona_name, persona_info in PERSONAS.items():
        prompt = f"""
Topic: {TOPIC}

Role: {persona_info['description']}

{persona_info['instruction']}

Provide 3-5 key arguments in a clear, structured format. Be specific and persuasive.
Keep your response concise (max 200 words).
"""

        print(f"  → Getting {persona_name} perspective...")
        response = call_llm(model, prompt)
        arguments[model][persona_name] = response
        time.sleep(0.5)  # Rate limiting

print("\n✓ Stage 1 complete! All arguments collected.")
print(f"Total responses: {len(MODELS) * len(PERSONAS)}")

STAGE 1: GENERATING ARGUMENTS FROM DIFFERENT PERSONAS

🤖 Querying gpt-4o-mini...
  → Getting optimist perspective...
  → Getting skeptic perspective...
  → Getting pragmatist perspective...

🤖 Querying claude-3-5-haiku-20241022...
  → Getting optimist perspective...
  → Getting skeptic perspective...
  → Getting pragmatist perspective...

🤖 Querying gemini-1.5-flash...
  → Getting optimist perspective...
  → Getting skeptic perspective...
  → Getting pragmatist perspective...

✓ Stage 1 complete! All arguments collected.
Total responses: 9


In [41]:
# =============================================================================
# CELL 6: Display Arguments
# =============================================================================
"""
View the arguments generated by each model/persona combination.
"""

print("\n" + "=" * 80)
print("COLLECTED ARGUMENTS")
print("=" * 80)

for model in MODELS:
    print(f"\n{'='*80}")
    print(f"MODEL: {model}")
    print(f"{'='*80}")

    for persona_name in PERSONAS.keys():
        print(f"\n--- {persona_name.upper()} ---")
        print(arguments[model][persona_name])
        print()


COLLECTED ARGUMENTS

MODEL: gpt-4o-mini

--- OPTIMIST ---
ERROR: litellm.RateLimitError: RateLimitError: OpenAIException - You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.


--- SKEPTIC ---
ERROR: litellm.RateLimitError: RateLimitError: OpenAIException - You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.


--- PRAGMATIST ---
ERROR: litellm.RateLimitError: RateLimitError: OpenAIException - You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.


MODEL: claude-3-5-haiku-20241022

--- OPTIMIST ---
ERROR: litellm.BadRequestError: AnthropicException - {"type":"error","error":{

In [21]:
# =============================================================================
# CELL 7: Stage 2 - Create Comparison Tables
# =============================================================================
"""
Ask each model to analyze all arguments and create structured comparison tables.
"""

print("=" * 80)
print("STAGE 2: CREATING COMPARISON TABLES")
print("=" * 80)

# Compile all arguments for analysis
all_arguments_text = "\n\n".join([
    f"=== {model} - {persona} ===\n{arg}"
    for model, personas in arguments.items()
    for persona, arg in personas.items()
])

comparison_prompt = f"""
Topic: {TOPIC}

Here are arguments from multiple perspectives:

{all_arguments_text}

Analyze these arguments and create a structured assessment with these criteria:

1. VALENCE: Categorize each major point as Positive (Pro), Negative (Con), or Neutral
2. IMPACT: Rate the potential impact as Small, Medium, or Large
3. CONFIDENCE: Rate your confidence in the claim as Low, Medium, or High

Provide your analysis in JSON format:
{{
    "points": [
        {{
            "description": "brief point description",
            "valence": "positive/negative/neutral",
            "impact": "small/medium/large",
            "confidence": "low/medium/high"
        }}
    ]
}}

Identify 5-8 key points total. Return ONLY the JSON, no additional text.
"""

comparisons = {}

for model in MODELS:
    print(f"\n🤖 Getting comparison from {model}...")
    response = call_llm(model, comparison_prompt)

    try:
        comparisons[model] = extract_json_from_response(response)
        print(f"  ✓ Extracted {len(comparisons[model]['points'])} points")
    except Exception as e:
        print(f"  ✗ Error parsing response: {e}")
        comparisons[model] = {"points": [], "error": str(e)}

    time.sleep(0.5)

print("\n✓ Stage 2 complete! Comparison tables created.")


STAGE 2: CREATING COMPARISON TABLES

🤖 Getting comparison from gpt-4o-mini...
  ✗ Error parsing response: Could not extract JSON from response

🤖 Getting comparison from claude-3-5-haiku-20241022...
  ✗ Error parsing response: 'points'


23:37:53 - LiteLLM:ERROR: vertex_llm_base.py:494 - Failed to load vertex credentials. Check to see if credentials containing partial/invalid information. Error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7b36ab327d10>)
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 140, in _refresh_token
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 107, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 342, in get_service_account_info
    return get(request, path, params={"r


🤖 Getting comparison from gemini-1.5-flash...


23:37:55 - LiteLLM:ERROR: vertex_llm_base.py:494 - Failed to load vertex credentials. Check to see if credentials containing partial/invalid information. Error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7b36a9c0ed80>)
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 140, in _refresh_token
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 107, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 342, in get_service_account_info
    return get(request, path, params={"r

  ✗ Error parsing response: 'points'

✓ Stage 2 complete! Comparison tables created.


In [22]:
# =============================================================================
# CELL 8: Display Comparison Tables
# =============================================================================
"""
View the structured comparisons from each model.
"""

print("\n" + "=" * 80)
print("COMPARISON TABLES")
print("=" * 80)

for model, data in comparisons.items():
    print(f"\n{'='*80}")
    print(f"MODEL: {model}")
    print(f"{'='*80}\n")

    if "error" in data:
        print(f"ERROR: {data['error']}")
        continue

    df = pd.DataFrame(data['points'])
    if not df.empty:
        print(df.to_string(index=False))
    else:
        print("No points extracted")
    print()


COMPARISON TABLES

MODEL: gpt-4o-mini

ERROR: Could not extract JSON from response

MODEL: claude-3-5-haiku-20241022

ERROR: 'points'

MODEL: gemini-1.5-flash

ERROR: 'points'


In [23]:
# =============================================================================
# CELL 9: Stage 3 - Quantify Assessments
# =============================================================================
"""
Convert categorical assessments to numerical scores (1-3 scale).
"""

print("=" * 80)
print("STAGE 3: QUANTIFYING ASSESSMENTS")
print("=" * 80)

# Mapping for conversion
VALENCE_MAP = {"positive": 3, "neutral": 2, "negative": 1}
IMPACT_MAP = {"large": 3, "medium": 2, "small": 1}
CONFIDENCE_MAP = {"high": 3, "medium": 2, "low": 1}

quantified_data = []

for model, data in comparisons.items():
    if "error" in data or not data.get('points'):
        continue

    for point in data['points']:
        quantified_data.append({
            'model': model,
            'description': point['description'],
            'valence': point['valence'],
            'valence_score': VALENCE_MAP.get(point['valence'].lower(), 2),
            'impact': point['impact'],
            'impact_score': IMPACT_MAP.get(point['impact'].lower(), 2),
            'confidence': point['confidence'],
            'confidence_score': CONFIDENCE_MAP.get(point['confidence'].lower(), 2)
        })

df_quantified = pd.DataFrame(quantified_data)

print(f"\n✓ Quantified {len(df_quantified)} points")
print("\nSample of quantified data:")
print(df_quantified.head(10).to_string(index=False))


STAGE 3: QUANTIFYING ASSESSMENTS

✓ Quantified 0 points

Sample of quantified data:
Empty DataFrame
Columns: []
Index: []


In [24]:
# =============================================================================
# CELL 10: Stage 4 - Aggregate Scores
# =============================================================================
"""
Calculate average scores across all models.
"""

print("\n" + "=" * 80)
print("STAGE 4: AGGREGATING SCORES")
print("=" * 80)

if not df_quantified.empty:
    # Calculate overall averages
    avg_scores = {
        'avg_valence': df_quantified['valence_score'].mean(),
        'avg_impact': df_quantified['impact_score'].mean(),
        'avg_confidence': df_quantified['confidence_score'].mean()
    }

    print("\n📊 OVERALL AVERAGE SCORES (1-3 scale):")
    print(f"  Valence (1=Negative, 2=Neutral, 3=Positive): {avg_scores['avg_valence']:.2f}")
    print(f"  Impact (1=Small, 2=Medium, 3=Large): {avg_scores['avg_impact']:.2f}")
    print(f"  Confidence (1=Low, 2=Medium, 3=High): {avg_scores['avg_confidence']:.2f}")

    # Calculate by model
    print("\n📊 SCORES BY MODEL:")
    model_scores = df_quantified.groupby('model')[['valence_score', 'impact_score', 'confidence_score']].mean()
    print(model_scores.to_string())

    # Valence distribution
    print("\n📊 VALENCE DISTRIBUTION:")
    valence_counts = df_quantified['valence'].value_counts()
    print(valence_counts.to_string())

else:
    print("⚠️  No quantified data available for aggregation")


STAGE 4: AGGREGATING SCORES
⚠️  No quantified data available for aggregation


In [25]:
# =============================================================================
# CELL 11: Stage 5 - Voting and Conflict Resolution
# =============================================================================
"""
Resolve disagreements through voting mechanisms.
"""

print("\n" + "=" * 80)
print("STAGE 5: CONFLICT RESOLUTION VIA VOTING")
print("=" * 80)

if not df_quantified.empty:
    # Method 1: Majority Vote on Overall Stance
    print("\n🗳️  METHOD 1: MAJORITY VOTE ON OVERALL STANCE")
    valence_votes = df_quantified['valence'].value_counts()
    majority_stance = valence_votes.idxmax()
    print(f"Majority stance: {majority_stance.upper()}")
    print(f"Vote distribution: {valence_votes.to_dict()}")

    # Method 2: Weighted Vote (by confidence)
    print("\n🗳️  METHOD 2: CONFIDENCE-WEIGHTED VOTE")
    weighted_sum = (df_quantified['valence_score'] * df_quantified['confidence_score']).sum()
    total_confidence = df_quantified['confidence_score'].sum()
    weighted_avg = weighted_sum / total_confidence if total_confidence > 0 else 0

    if weighted_avg > 2.3:
        weighted_stance = "POSITIVE"
    elif weighted_avg < 1.7:
        weighted_stance = "NEGATIVE"
    else:
        weighted_stance = "NEUTRAL"

    print(f"Weighted stance: {weighted_stance}")
    print(f"Weighted average: {weighted_avg:.2f}")

    # Method 3: Model Consensus
    print("\n🗳️  METHOD 3: MODEL CONSENSUS")
    model_stances = df_quantified.groupby('model')['valence_score'].mean()

    consensus_threshold = 0.5  # How close models need to be
    std_dev = model_stances.std()

    if std_dev < consensus_threshold:
        print(f"✓ HIGH CONSENSUS (std dev: {std_dev:.2f})")
    else:
        print(f"⚠️  LOW CONSENSUS (std dev: {std_dev:.2f})")

    print("\nModel stances:")
    for model, score in model_stances.items():
        stance = "POSITIVE" if score > 2.3 else "NEGATIVE" if score < 1.7 else "NEUTRAL"
        print(f"  {model}: {score:.2f} ({stance})")

    # Method 4: High-Confidence Points Only
    print("\n🗳️  METHOD 4: HIGH-CONFIDENCE POINTS ONLY")
    high_conf = df_quantified[df_quantified['confidence_score'] >= 2.5]
    if len(high_conf) > 0:
        hc_avg_valence = high_conf['valence_score'].mean()
        hc_stance = "POSITIVE" if hc_avg_valence > 2.3 else "NEGATIVE" if hc_avg_valence < 1.7 else "NEUTRAL"
        print(f"High-confidence stance: {hc_stance}")
        print(f"Based on {len(high_conf)} high-confidence points")
    else:
        print("No high-confidence points found")

else:
    print("⚠️  No data available for voting")



STAGE 5: CONFLICT RESOLUTION VIA VOTING
⚠️  No data available for voting


In [26]:
# =============================================================================
# CELL 12: Final Summary Report
# =============================================================================
"""
Generate a comprehensive summary of the multi-agent analysis.
"""

print("\n" + "=" * 80)
print("FINAL SUMMARY REPORT")
print("=" * 80)

print(f"\n📋 TOPIC: {TOPIC}")
print(f"🤖 MODELS CONSULTED: {len(MODELS)}")
print(f"👥 PERSONAS USED: {len(PERSONAS)}")
print(f"💬 TOTAL POINTS ANALYZED: {len(df_quantified)}")

if not df_quantified.empty:
    print("\n" + "-" * 80)
    print("CONSENSUS FINDINGS:")
    print("-" * 80)

    print(f"\n1. Overall Sentiment: {majority_stance.upper()}")
    print(f"   - {valence_votes[majority_stance]} out of {len(df_quantified)} points support this")

    print(f"\n2. Expected Impact: {df_quantified['impact'].mode().values[0].upper()}")
    print(f"   - Average impact score: {avg_scores['avg_impact']:.2f}/3.0")

    print(f"\n3. Confidence Level: {df_quantified['confidence'].mode().values[0].upper()}")
    print(f"   - Average confidence score: {avg_scores['avg_confidence']:.2f}/3.0")

    print(f"\n4. Model Agreement: {'HIGH' if std_dev < 0.5 else 'MODERATE' if std_dev < 1.0 else 'LOW'}")
    print(f"   - Standard deviation: {std_dev:.2f}")

    print("\n" + "-" * 80)
    print("RECOMMENDATION:")
    print("-" * 80)

    # Generate a final recommendation based on all factors
    if weighted_avg > 2.3 and avg_scores['avg_confidence'] > 2.0:
        recommendation = "STRONGLY SUPPORT"
    elif weighted_avg > 2.0:
        recommendation = "CAUTIOUSLY SUPPORT"
    elif weighted_avg < 1.7 and avg_scores['avg_confidence'] > 2.0:
        recommendation = "STRONGLY OPPOSE"
    elif weighted_avg < 2.0:
        recommendation = "CAUTIOUSLY OPPOSE"
    else:
        recommendation = "REMAIN NEUTRAL / GATHER MORE DATA"

    print(f"\n{recommendation}")
    print(f"\nThis recommendation is based on:")
    print(f"  • Weighted valence score: {weighted_avg:.2f}")
    print(f"  • Average confidence: {avg_scores['avg_confidence']:.2f}")
    print(f"  • Model consensus: {std_dev:.2f} std dev")

else:
    print("\n⚠️  Insufficient data for final recommendation")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)


FINAL SUMMARY REPORT

📋 TOPIC: What is the tallest tree that ever existed?
🤖 MODELS CONSULTED: 3
👥 PERSONAS USED: 3
💬 TOTAL POINTS ANALYZED: 0

⚠️  Insufficient data for final recommendation

ANALYSIS COMPLETE
